In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
import re
import random

In [2]:
import pandas as pd


In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [4]:
driver = webdriver.Chrome(chromedriver)

In [5]:
rows=[]

In [6]:
#url1 = 'https://shop.rebag.com/collections/all-bags?page=1&_=pf&pf_st_availability_hidden=true&pf_t_categories=bc-filter-Bags'

In [7]:
how_many_pages = 4
for i in range(1, 20, 1):
    url_page = 'https://shop.rebag.com/collections/all-bags?page=' + str(i) + '&_=pf&pf_st_availability_hidden=true&pf_t_categories=bc-filter-Bags'
    driver.get(url_page)
    soup1 = BeautifulSoup(driver.page_source, 'html.parser')
    rows_one_page = soup1.find_all(class_='product-caption') #.find('a')
    rows.append(rows_one_page)
    time.sleep(.5+2*random.random())


In [8]:
links_bags = []
for row in rows:
    for r in row:
        links_bags.append(r.find('a')['href'])

In [9]:
len(links_bags)

912

In [10]:
links_bags

['/collections/all-bags/products/handbags-chanel-boy-flap-bag-quilted-caviar-old-medium5695319',
 '/collections/all-bags/products/handbags-chanel-coco-top-handle-bag-quilted-caviar-medium678761',
 '/collections/all-bags/products/handbags-christian-dior-book-tote-embroidered-canvas-small662631',
 '/collections/all-bags/products/handbags-christian-dior-saddle-handbag-oblique-canvas-medium627171',
 '/collections/all-bags/products/handbags-prada-promenade-bag-saffiano-leather-mini5709211',
 '/collections/all-bags/products/handbags-christian-dior-saddle-handbag-oblique-canvas-mini685261',
 '/collections/all-bags/products/handbags-fendi-baguette-nm-bag-zucca-lycra-medium65203497',
 '/collections/all-bags/products/handbags-balenciaga-city-rubber-studs-bag-leather-mini5386756',
 '/collections/all-bags/products/handbags-louis-vuitton-pochette-accessoires-damier-mini615452',
 '/collections/all-bags/products/handbags-louis-vuitton-multi-pochette-accessoires-monogram-canvas-619491',
 '/collections

In [11]:
def money_to_int(moneystring):
    if moneystring == 0:
        return None
    moneystring = moneystring.replace('$', '').replace(',', '').replace('USD', '').replace('Est.', '').replace('Retail', '')
    return int(moneystring)

In [12]:
def size_to_float(sizestring):
#Adapt volume of bag to float format

    if sizestring == 0:
        return None
    volume = 1
    sizestring = sizestring.replace('W', '').replace('H', '').replace('x', '')
    size_list = sizestring.split('"')
    if len(size_list) < 3:
        return None
    for i,el in enumerate(size_list):
        if i < 3:
            el = float(el.strip())
            volume = volume * el
            
    return volume


In [13]:
def get_details_value(soup, field_name):
    
    '''Grab a value from Box Office Mojo HTML
    
    Takes a string attribute of a movie on the page and returns the string in
    the next sibling object (the value for that attribute) or None if nothing is found.
    '''
    
    obj = soup.find(text=field_name)
    
    if not obj: 
        return None
    
    # this works for most of the values
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text 
    else:
        return None

In [14]:
def get_bags_dict(link):
    '''
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - domestic gross
        - runtime 
        - MPAA rating
        - full release date
    Return information as a dictionary.
    '''
    
    base_url = 'https://shop.rebag.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    headers = ['Title', 'Brand', 'Price', 'Condition', 'Retail_Price', 'Volume', 'Exterior Color',
               'Exterior Material', 'Interior Color', 'Interior Material', 'Accessories', 'Ac_Count']
    
    
    #title
    title_string = soup.find('title').text
    title = title_string.split('-')[0].strip()
    title = title.rsplit(' ', 1)[0]
    print(title)
    
    if soup.find(class_='pdp__designer'):
        brand = soup.find(class_='pdp__designer').text
    else:
        brand = None
        
    if soup.find(itemprop='price'):
        raw_price = soup.find(itemprop='price').text 
        price = money_to_int(raw_price)
        #price = raw_price
    else:
        price=None
    #print(price)
    
    if soup.find(class_='pdp__condition-item--selected'):
        condition = soup.find(class_='pdp__condition-item--selected').text
    else:
        condition=None
    
    #!!! Change for good form!
    if soup.find(class_='pdp__retail-price'):
        raw_retail_price = soup.find(class_='pdp__retail-price').text
        retail_price = money_to_int(raw_retail_price)
        #retail_price = raw_retail_price
    else:
        retail_price=None
        
    ex_color = get_details_value(soup,'Exterior Color')
    #print(ex_color)
    
    ex_material = get_details_value(soup,'Exterior Material')
    #print(ex_material)
    
    int_color = get_details_value(soup,'Interior Color')
    
    int_material = get_details_value(soup,'Interior Material')
    
    # NEW!!!
    # Volume
    
    raw_volume = get_details_value(soup, 'SIZE AND FIT')
    if (raw_volume):
        volume = size_to_float(raw_volume)
    else:
        volume = None
    
    
    #Accessories
    accessories=[]
    for el_class in soup.find_all(class_="pdp__group-title"):
        if el_class.text == 'Accessories':
            for element in el_class.findNext().find_all('div'):
                accessories.append(element.text)
    
    if len(accessories) == 0:
        accessories_count = None
    elif len(accessories) == 1 and accessories[0]=='No accessories':
        accessories_count = 0
    else:
        accessories_count = len(accessories)
    #
    
    bags_dict = dict(zip(headers, [title,
                                brand,
                                price,
                                condition,
                                retail_price,
                                volume,
                                ex_color,
                                ex_material,
                                int_color, 
                                int_material,
                                accessories,
                                accessories_count]))

    return bags_dict
#, price

In [15]:
links_bags[0:10]

['/collections/all-bags/products/handbags-chanel-boy-flap-bag-quilted-caviar-old-medium5695319',
 '/collections/all-bags/products/handbags-chanel-coco-top-handle-bag-quilted-caviar-medium678761',
 '/collections/all-bags/products/handbags-christian-dior-book-tote-embroidered-canvas-small662631',
 '/collections/all-bags/products/handbags-christian-dior-saddle-handbag-oblique-canvas-medium627171',
 '/collections/all-bags/products/handbags-prada-promenade-bag-saffiano-leather-mini5709211',
 '/collections/all-bags/products/handbags-christian-dior-saddle-handbag-oblique-canvas-mini685261',
 '/collections/all-bags/products/handbags-fendi-baguette-nm-bag-zucca-lycra-medium65203497',
 '/collections/all-bags/products/handbags-balenciaga-city-rubber-studs-bag-leather-mini5386756',
 '/collections/all-bags/products/handbags-louis-vuitton-pochette-accessoires-damier-mini615452',
 '/collections/all-bags/products/handbags-louis-vuitton-multi-pochette-accessoires-monogram-canvas-619491']

List of all bags, let's try open all pages from this page

In [16]:
bags_page_info_list = []
for link in links_bags:
    bags_page_info_list.append(get_bags_dict(link))
    time.sleep(5+2*random.random())

Chanel Boy Flap Bag Quilted Caviar Old Medium
Chanel Coco Top Handle Bag Quilted Caviar Medium
Christian Dior Book Tote Embroidered Canvas Small
Christian Dior Saddle Handbag Oblique Canvas Medium
Prada Promenade Bag Saffiano Leather Mini
Christian Dior Saddle Handbag Oblique Canvas Mini
Fendi Baguette NM Bag Zucca Lycra Medium
Balenciaga City Rubber Studs Bag Leather Mini
Louis Vuitton Pochette Accessoires Damier Mini
Louis Vuitton Multi Pochette Accessoires Monogram Canvas
Hermes Kelly Handbag Natural Sable Clemence with Gold Hardware 32
Chanel Flat Handle Boy Flap Bag Quilted Lambskin Old Medium
Chanel Deauville Tote Lurex Canvas Small
Christian Dior Ultra Matte Lady Dior Bag Cannage Quilt Calfskin Medium
Louis Vuitton Multi Pochette Accessoires Monogram Canvas
Chanel Coco Envelope Flap Bag Chevron Leather Medium
Christian Dior Diorama Wallet on Chain Satin
Hermes Vintage Cadena Top Handle Bag Box Calf
Saint Laurent Classic Monogram Tassel Chain Wallet Leather
Saint Laurent Rive Gau

KeyboardInterrupt: 

In [ ]:
bags_page_info_list

In [ ]:
bags_page_info = pd.DataFrame(bags_page_info_list)  #convert list of dict to df
bags_page_info.set_index('Title', inplace=True)

bags_page_info

In [ ]:
bags_page_info.info()

In [ ]:
bags_page_info.to_csv('bags_page_info.csv')

In [ ]:
bags_page_info